In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com'

# Load data

In [ ]:
LCLUC2000 = ee.Image('projects/glad/GLCLU2020/LCLUC_2000')
LCLUC2020 = ee.Image('projects/glad/GLCLU2020/LCLUC_2020')
LCLUCscale = LCLUC2000.projection().nominalScale()

def simplifyLCLUC(im):
    im = (im.where(im.eq(0),0)
    .where(im.gte(1).And(im.lte(24)),1)
    .where(im.gte(25).And(im.lte(41)),2)
    .where(im.gte(42).And(im.lte(48)),3)
    .where(im.gte(100).And(im.lte(124)),4)
    .where(im.gte(125).And(im.lte(148)),5)
    .where(im.gte(200).And(im.lte(207)),6)
    ##.where(im.eq(254),6)
    .where(im.eq(241),7)
    .where(im.eq(244),8)
    .where(im.eq(250),9)
    .where(im.eq(255),10))
    return im

LCLUC2000 = simplifyLCLUC(LCLUC2000)
LCLUC2020 = simplifyLCLUC(LCLUC2020)

LANDCLASSES = {
    0: {'name': 'bare ground', 'is_habitat': 0, 'color': 'FEFECC'},
    1: {'name': 'short vegetation', 'is_habitat': 1, 'color': 'B9B91E'},
    2: {'name': 'forest', 'is_habitat': 1, 'color': '347834'},
    3: {'name': 'tall forest', 'is_habitat': 1, 'color': '0D570D'},
    4: {'name': 'wetland - short vegetation', 'is_habitat': 1, 'color': '88CAAD'},
    5: {'name': 'wetland - forest', 'is_habitat': 1, 'color': '589558'},
    6: {'name': 'water', 'is_habitat': 1, 'color': '6BAED6'},
    7: {'name': 'snow/ice', 'is_habitat': 0, 'color': 'ACD1E8'},
    8: {'name': 'cropland', 'is_habitat': 0, 'color': 'FFF183'},
    9: {'name': 'built-up', 'is_habitat': 0, 'color': 'E8765D'}
}

habitat_2000 = LCLUC2000.remap(list(LANDCLASSES.keys()), [int(LANDCLASSES[i]['is_habitat']) for i in LANDCLASSES], 0)
habitat_2020 = LCLUC2020.remap(list(LANDCLASSES.keys()), [int(LANDCLASSES[i]['is_habitat']) for i in LANDCLASSES], 0)

class_00 = habitat_2000.eq(0).multiply(habitat_2020.eq(0)) # 0: never habitat
class_01 = habitat_2000.multiply(-1).add(1).multiply(habitat_2020) # 1: became habitat between 2000 and 2020
class_10 = habitat_2020.multiply(-1).add(1).multiply(habitat_2000) # 10: became nonhabitat between 2000 and 2020
class_11 = habitat_2000.eq(1).multiply(habitat_2020.eq(1)) # 11: was always habitat
habitatchange = class_01.updateMask(class_01.eq(1)).where(1,1).blend(class_10.updateMask(class_10.eq(1)).where(1,10))

Map = geemap.Map(height="400px")
Map.addLayer(habitat_2000,{'min':0,'max':1},"habitat_2000")
Map.addLayer(habitat_2020,{'min':0,'max':1},"habitat_2020")
Map.addLayer(habitatchange,{'min':1,'max':10},"change")
Map

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir+'/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract land cover by city

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir+'/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image as GeoTIFF to Google Drive 
    geemap.ee_export_image_to_drive(
        LCLUC2000.toByte(), # use toByte() to reduce file size
        description = boundary_id + '-GLADlandcover-2000',
        folder='data', 
        scale=100, #LCLUCscale or 100
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )
    
    geemap.ee_export_image_to_drive(
        LCLUC2020.toByte(), # use toByte() to reduce file size
        description = boundary_id + '-GLADlandcover-2020',
        folder='data', 
        scale=100, #LCLUCscale or 100
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )
    
    geemap.ee_export_image_to_drive(
        habitatchange.toByte(), # use toByte() to reduce file size
        description = boundary_id + '-habitatchange-2000to2020',
        folder='data', 
        scale=100, #LCLUCscale or 100
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id + '-GLADlandcover-2000.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/GLAD-ARD/v_0/' + boundary_id + '-GLADlandcover-2000-100m.tif',
                               ExtraArgs={'ACL':'public-read'})

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id + '-GLADlandcover-2020.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/GLAD-ARD/v_0/' + boundary_id + '-GLADlandcover-2020-100m.tif',
                               ExtraArgs={'ACL':'public-read'})

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id + '-habitatchange-2000to2020.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/GLAD-ARD/v_0/' + boundary_id + '-habitatchange-2000to2020-100m.tif',
                               ExtraArgs={'ACL':'public-read'})

# Visualize data

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=8)

In [ ]:
Map.addLayer(greenmask),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Vegetation cover',True,1)